In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.sparse import csr_matrix, hstack, save_npz, load_npz

/home/maxou1909/miniconda3/envs/ml/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# def wrap_create_sparse_df(df, n_cols, n_rows, col_to_sparse_df, value_to_sparse_df, row_id):
#     df_sparse = pd.DataFrame()
#     for i in tqdm(range(n_cols)):
#         a = np.empty((n_rows))
#         a[:] = 0

#         df_product = df[df[col_to_sparse_df] == i+1]
#         for c in df_product[value_to_sparse_df].unique():
#             df_p = df_product[df_product["count"] == c]
#             a[df_p[row_id].to_list()] = c
#         df_sparse[str(i)] = pd.arrays.SparseArray(a, dtype=float, fill_value=0)
#     return df_sparse

In [3]:
# def create_sparse_df(df, col_to_sparse_df, row_id):
#     value_to_sparse_df = "count"
#     df_exploded = df.explode(col_to_sparse_df)   # in case the column contains a list
#     df_exploded[value_to_sparse_df] = 1          # this column is used to keep track of the sparse value
    
#     df_exploded_grouped = df_exploded.groupby([col_to_sparse_df, row_id])[value_to_sparse_df].sum().reset_index()
#     n_cols = df_exploded_grouped[col_to_sparse_df].max()
#     n_rows = df_exploded_grouped[row_id].max() + 1
    
#     df_sparse = wrap_create_sparse_df(df_exploded_grouped, n_cols, n_rows, col_to_sparse_df, value_to_sparse_df, row_id)
#     return df_sparse

In [4]:
# df_product_to_sparse = df_order_products[["order_id", "product_id"]]
# df_sparse_product = create_sparse_df(df_product_to_sparse, col_to_sparse_df="product_id", row_id="order_id")
# del df_product_to_sparse
# df_sparse_product

In [5]:
df_order_products = pd.read_csv("../data/raw/order_products__prior.csv")
df_products = pd.read_csv("../data/raw/products.csv")

In [6]:
df_order_products["count"] = 1
df = df_order_products.groupby(["product_id"])["count"].sum().reset_index()
df["percentage"] = df["count"]/len(df_order_products)
df = df.sort_values("percentage", ascending=False)
df = df.iloc[0:10, :]
most_sold_products = df["product_id"].to_list()

In [7]:
df_orders = df_order_products[df_order_products["product_id"].isin(most_sold_products)]
df_orders

,order_id,product_id,add_to_cart_order,reordered,count
12,3,21903,4,1,1
30,5,13176,1,1,1
51,5,47209,22,0,1
77,10,24852,1,1,1
80,10,47766,4,1,1
...,...,...,...,...,...
32434445,3421077,27845,4,0,1
32434447,3421078,24852,2,1,1
32434456,3421080,27845,1,1,1
32434474,3421082,16797,3,0,1


In [8]:
df_orders = df_orders.merge(df_products, on="product_id")
df_orders = df_orders[["order_id", "product_name"]]
df_orders

,order_id,product_name
0,3,Organic Baby Spinach
1,26,Organic Baby Spinach
2,31,Organic Baby Spinach
3,39,Organic Baby Spinach
4,56,Organic Baby Spinach
...,...,...
2323153,3421024,Limes
2323154,3421030,Limes
2323155,3421039,Limes
2323156,3421053,Limes


In [9]:
df_orders = df_orders[df_orders["order_id"] < 10000]
df_orders

,order_id,product_name
0,3,Organic Baby Spinach
1,26,Organic Baby Spinach
2,31,Organic Baby Spinach
3,39,Organic Baby Spinach
4,56,Organic Baby Spinach
...,...,...
2182913,9876,Limes
2182914,9928,Limes
2182915,9948,Limes
2182916,9960,Limes


In [10]:
df2 = pd.crosstab(df_orders['order_id'], df_orders['product_name'])
df2

product_name,Bag of Organic Bananas,Banana,Large Lemon,Limes,Organic Avocado,Organic Baby Spinach,Organic Hass Avocado,Organic Strawberries,Organic Whole Milk,Strawberries
order_id,,,,,,,,,,
3,0,0,0,0,0,1,0,0,0,0
5,1,0,0,0,0,0,1,0,0,0
10,0,1,0,0,1,0,0,1,0,0
14,0,0,0,0,0,0,0,0,1,0
18,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9989,0,0,0,0,0,0,1,0,0,0
9990,0,0,0,0,1,0,0,0,0,0
9993,1,0,0,0,0,0,0,0,0,0


In [11]:
def encode(item_freq):
    res = 0
    if item_freq > 0:
        res = 1
    return res
    
basket_input = df2.applymap(encode)

In [16]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket_input, min_support=0.001, max_len=2, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift")

rules.head()

/home/maxou1909/miniconda3/envs/ml/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/maxou1909/miniconda3/envs/ml/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Organic Baby Spinach),(Bag of Organic Bananas),0.155485,0.257706,0.033772,0.217201,0.842825,-0.006298,0.948256,-0.180879
1,(Bag of Organic Bananas),(Organic Baby Spinach),0.257706,0.155485,0.033772,0.131047,0.842825,-0.006298,0.971876,-0.200786
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.138486,0.257706,0.041251,0.297872,1.155860,0.005562,1.057206,0.156519
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.257706,0.138486,0.041251,0.160070,1.155860,0.005562,1.025698,0.181658
4,(Bag of Organic Bananas),(Organic Strawberries),0.257706,0.169538,0.036718,0.142480,0.840405,-0.006973,0.968447,-0.203716


In [17]:
rules.sort_values(["support", "confidence","lift"],axis = 0, ascending = False).head(8)

/home/maxou1909/miniconda3/envs/ml/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.138486,0.257706,0.041251,0.297872,1.155860,0.005562,1.057206,0.156519
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.257706,0.138486,0.041251,0.160070,1.155860,0.005562,1.025698,0.181658
5,(Organic Strawberries),(Bag of Organic Bananas),0.169538,0.257706,0.036718,0.216578,0.840405,-0.006973,0.947501,-0.186113
4,(Bag of Organic Bananas),(Organic Strawberries),0.257706,0.169538,0.036718,0.142480,0.840405,-0.006973,0.968447,-0.203716
0,(Organic Baby Spinach),(Bag of Organic Bananas),0.155485,0.257706,0.033772,0.217201,0.842825,-0.006298,0.948256,-0.180879
1,(Bag of Organic Bananas),(Organic Baby Spinach),0.257706,0.155485,0.033772,0.131047,0.842825,-0.006298,0.971876,-0.200786
8,(Organic Avocado),(Banana),0.118767,0.318676,0.033318,0.280534,0.880311,-0.004530,0.946986,-0.133664
9,(Banana),(Organic Avocado),0.318676,0.118767,0.033318,0.104552,0.880311,-0.004530,0.984125,-0.166358


In [18]:
rules.sort_values(["lift"],axis = 0, ascending = False).head(8)

/home/maxou1909/miniconda3/envs/ml/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
15,(Limes),(Large Lemon),0.087715,0.097688,0.015186,0.173127,1.772238,0.006617,1.091233,0.477638
14,(Large Lemon),(Limes),0.097688,0.087715,0.015186,0.155452,1.772238,0.006617,1.080205,0.482917
21,(Limes),(Organic Avocado),0.087715,0.118767,0.014053,0.160207,1.348916,0.003635,1.049345,0.283534
20,(Organic Avocado),(Limes),0.118767,0.087715,0.014053,0.118321,1.348916,0.003635,1.034712,0.293525
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.138486,0.257706,0.041251,0.297872,1.155860,0.005562,1.057206,0.156519
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.257706,0.138486,0.041251,0.160070,1.155860,0.005562,1.025698,0.181658
17,(Organic Avocado),(Large Lemon),0.118767,0.097688,0.012919,0.108779,1.113530,0.001317,1.012444,0.115696
16,(Large Lemon),(Organic Avocado),0.097688,0.118767,0.012919,0.132251,1.113530,0.001317,1.015539,0.112993


In [19]:
rules

/home/maxou1909/miniconda3/envs/ml/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Organic Baby Spinach),(Bag of Organic Bananas),0.155485,0.257706,0.033772,0.217201,0.842825,-0.006298,0.948256,-0.180879
1,(Bag of Organic Bananas),(Organic Baby Spinach),0.257706,0.155485,0.033772,0.131047,0.842825,-0.006298,0.971876,-0.200786
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.138486,0.257706,0.041251,0.297872,1.155860,0.005562,1.057206,0.156519
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.257706,0.138486,0.041251,0.160070,1.155860,0.005562,1.025698,0.181658
4,(Bag of Organic Bananas),(Organic Strawberries),0.257706,0.169538,0.036718,0.142480,0.840405,-0.006973,0.968447,-0.203716
5,(Organic Strawberries),(Bag of Organic Bananas),0.169538,0.257706,0.036718,0.216578,0.840405,-0.006973,0.947501,-0.186113
6,(Bag of Organic Bananas),(Organic Whole Milk),0.257706,0.082276,0.017226,0.066843,0.812423,-0.003977,0.983461,-0.237249
7,(Organic Whole Milk),(Bag of Organic Bananas),0.082276,0.257706,0.017226,0.209366,0.812423,-0.003977,0.938859,-0.201014
8,(Organic Avocado),(Banana),0.118767,0.318676,0.033318,0.280534,0.880311,-0.004530,0.946986,-0.133664
9,(Banana),(Organic Avocado),0.318676,0.118767,0.033318,0.104552,0.880311,-0.004530,0.984125,-0.166358
